# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace,Experiment,Dataset
from azureml.core.compute import ComputeTarget,AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.utilities import get_primary_metrics

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'exp-heart1'

experiment=Experiment(ws, experiment_name)

In [3]:
try:
    cluster=ComputeTarget(workspace=ws,name="compute-1")
    print('cluster exists')
except:
    config =AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cluster =ComputeTarget.create(ws,"compute-1",config)
    
cluster.wait_for_completion()
    

Provisioning operation finished, operation "Succeeded"


In [4]:
heart_data = Dataset.get_by_name(ws,name="heartfailure")
heart_data.take(5).to_pandas_dataframe()

age  anaemia  creatinine_phosphokinase  diabetes  ejection_fraction  \
0  75.0        0                       582         0                 20   
1  55.0        0                      7861         0                 38   
2  65.0        0                       146         0                 20   
3  50.0        1                       111         0                 20   
4  65.0        1                       160         1                 20   

   high_blood_pressure  platelets  serum_creatinine  serum_sodium  sex  \
0                    1  265000.00               1.9           130    1   
1                    0  263358.03               1.1           136    1   
2                    0  162000.00               1.3           129    1   
3                    0  210000.00               1.9           137    1   
4                    0  327000.00               2.7           116    0   

   smoking  time  DEATH_EVENT  
0        0     4            1  
1        0     6            1  
2        1     7            1  
3        0     7            1  
4        0     8            1

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "task": "classification", 
    "primary_metric": "AUC_weighted",
    "training_data": heart_data,
    "label_column_name": "DEATH_EVENT",
    "n_cross_validations": 3,
    "enable_early_stopping": True,
    "debug_log" :"automl_errors.log",
    "max_concurrent_iterations": 4,
    "compute_target": cluster,
    "iterations":100
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(**automl_settings)

In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [9]:
remote_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_d6715b1c-a1ca-46dc-ad82-97a9e190749c',
 'target': 'compute-1',
 'status': 'Completed',
 'startTimeUtc': '2021-04-22T09:47:35.284221Z',
 'endTimeUtc': '2021-04-22T10:15:18.150512Z',
 'properties': {'num_iterations': '100',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'compute-1',
  'AMLSettingsJsonString': '{"path":null,"name":"exp-heart1","subscription_id":"cdbe0b43-92a0-4715-838a-f2648cc7ad21","resource_group":"aml-quickstarts-143344","workspace_name":"quick-starts-ws-143344","region":"southcentralus","compute_target":"compute-1","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":100,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"test_size":0.0,"validation_size":0.0,"n

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
best_auto_run,best_auto_model=remote_run.get_output()
best_auto_model._final_estimator

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('31',
                                           Pipeline(memory=None,
                                                    steps=[('standardscalerwrapper',
                                                            <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7f8dfc616518>),
                                                           ('xgboostclassifier',
                                                            XGBoostClassifier(base_score=0.5,
                                                                              booster='gbtree',
                                                                              colsample_bylevel=1,
                                                                              colsample_bynode=1,
                                                                              colsample_bytree=0.9...
                 

In [11]:
#TODO: Save the best model
import joblib
joblib.dump(best_auto_model,"automl_model.pkl")

['automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [23]:
from azureml.core.model import Model
model=Model.register(ws,model_path='./automl_model.pkl',model_name='heart-failure-prediction')

Registering model heart-failure-prediction


TODO: In the cell below, send a request to the web service you deployed to test it.

In [24]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
env =best_auto_run.get_environment()
inference_config=InferenceConfig(entry_script='score.py',environment=env)
deployment_config=AciWebservice.deploy_configuration(cpu_cores=1,memory_gb=1,enable_app_insights=True)
servicename='test-heart-service'

In [21]:
from azureml.core.webservice import LocalWebservice
local_config=LocalWebservice.deploy_configuration(port=9000)

In [37]:
service = Model.deploy(ws,'automldeploy',[model],inference_config,deployment_config,overwrite=True)
service.wait_for_deployment(True)
print("State: " + service.state)
print("Scoring URI: " + service.scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-22 12:28:03+00:00 Creating Container Registry if not exists.
2021-04-22 12:28:04+00:00 Registering the environment.
2021-04-22 12:28:05+00:00 Use the existing image.
2021-04-22 12:28:05+00:00 Generating deployment configuration.
2021-04-22 12:28:07+00:00 Submitting deployment to compute..
2021-04-22 12:28:11+00:00 Checking the status of deployment automldeploy..
2021-04-22 12:34:06+00:00 Checking the status of inference endpoint automldeploy.
Succeeded
ACI service creation operation finished, operation "Succeeded"
State: Healthy
Scoring URI: http://390f77a5-a05d-4d88-96ea-29075b6e9995.southcentralus.azurecontainer.io/score


TODO: In the cell below, print the logs of the web service and delete the service

In [27]:
print(service.get_logs())

2021-04-22T12:04:53,995835400+00:00 - gunicorn/run 
2021-04-22T12:04:54,013179600+00:00 - rsyslog/run 
2021-04-22T12:04:54,027667600+00:00 - iot-server/run 
2021-04-22T12:04:54,059607000+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [28]:
print(service.state)

Healthy


In [30]:
heart_df =heart_data.to_pandas_dataframe()
heart_df.columns

Index(['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time',
       'DEATH_EVENT'],
      dtype='object')

In [39]:
%run endpoint.py

{"result": [1, 1]}


In [ ]:
service.delete()